# MeVsMarket

## 1. Read data from MySQL
### 1.1. Import important packages

In [1]:
import pymysql
import pandas as pd
import decimal

### 1.2. Connect to MySQL database reconai

In [2]:
mydb = pymysql.connect(host='localhost',
                             user='root',
                             password='peacebro',database='reconai')
print(mydb)

### 1.3. Read data from database

In [3]:
sql_plantoactual='SELECT * FROM `plantoactual`;'
sql_marketplantoactual='SELECT * FROM `marketplantoactual`'

In [4]:
df_me_trial = pd.read_sql(sql_plantoactual, mydb)
df_me_trial.head()

,CPG Region,Retailer,Year,Week,Category,Brand,UPC,Plan numbers,Actual numbers
0,North,SuperMart,2017,1,Widgets,A,1234567890,"5,78,000.00","5,83,780.00"
1,East,SuperMart,2017,1,Widgets,A,1234567890,"6,53,000.00","6,59,530.00"
2,South,SuperMart,2017,1,Widgets,A,1234567890,"8,04,900.00","8,12,949.00"
3,West,SuperMart,2017,1,Widgets,A,1234567890,"6,57,800.00","6,64,378.00"
4,North,BigCo,2017,1,Widgets,A,1234567890,"1,98,400.00","2,00,384.00"


In [5]:
df_market=pd.read_sql(sql_marketplantoactual, mydb)
df_market.head()

,CPG Region,Retailer,Year,Week,Category,Plan numbers,Actual numbers
0,North,SuperMart,2017,1,Widgets,"10,22,500.00","10,00,000.00"
1,East,SuperMart,2017,1,Widgets,"44,11,650.00","45,00,000.00"
2,South,SuperMart,2017,1,Widgets,"49,12,716.00","49,20,000.00"
3,West,SuperMart,2017,1,Widgets,"73,51,370.00","71,50,000.00"
4,North,BigCo,2017,1,Widgets,"79,66,500.00","81,50,000.00"


## 2. Editing Dataframe for merge
### 2.1. Convert 'Plan numbers' and 'Actual numbers' column to float type

In [6]:
def sales_product(df):
    #df['date'] = pd.to_datetime(df.assign(day=1, month=1)[['Year', 'month', 'day']])+pd.to_timedelta(df.Week*7, unit='days')
    df['Plan numbers'] = df['Plan numbers'].str.replace(',', '')
    df['Plan numbers'] = df['Plan numbers'].astype(float)
    df['Actual numbers'] = df['Actual numbers'].str.replace(',', '')
    df['Actual numbers'] = df['Actual numbers'].astype(float)
sales_product(df_me_trial)

### 2.2. Summing UPC product rows together

In [7]:
def merge_upc(df):
    for numbers in ['Plan numbers','Actual numbers']:
        for year in [2017,2018]:
            for week in range(1,53):
                for ret in ['SuperMart','BigCo','Rx']:
                    for reg in ['North','East','South','West']:
                        ex=df[(df['Retailer']==ret) & (df['CPG Region']==reg) & 
                        (df['Year']==year) & (df['Week']==week)].sum()
                        ex[numbers]=round(ex[numbers], 2)
                        df.loc[(df['Retailer']==ret) & (df['CPG Region']==reg) & 
                    (df['Year']==year) & (df['Week']==week) & 
                    (df['Category']=='Widgets'),
                    numbers] = ex[numbers]
    return df
df_me_trial=merge_upc(df_me_trial)
df_me_trial.drop_duplicates(subset =["CPG Region","Retailer","Year","Week"], 
                     keep = "first", inplace = True)

In [8]:
df_me_trial.head()

,CPG Region,Retailer,Year,Week,Category,Brand,UPC,Plan numbers,Actual numbers
0,North,SuperMart,2017,1,Widgets,A,1234567890,707800.0,714878.0
1,East,SuperMart,2017,1,Widgets,A,1234567890,887100.0,895971.0
2,South,SuperMart,2017,1,Widgets,A,1234567890,2414700.0,2438847.0
3,West,SuperMart,2017,1,Widgets,A,1234567890,1292000.0,1304920.0
4,North,BigCo,2017,1,Widgets,A,1234567890,295500.0,298455.0


## 2. Merge dataframes
### 2.1. Rename 'Plan numbers' and 'Actual numbers' column in market dataframe

In [9]:
df_market=df_market.rename(columns={'Plan numbers': 'Market Plan numbers'})
df_market=df_market.rename(columns={'Actual numbers': 'Market Actual numbers'})
df_market.head()

,CPG Region,Retailer,Year,Week,Category,Market Plan numbers,Market Actual numbers
0,North,SuperMart,2017,1,Widgets,"10,22,500.00","10,00,000.00"
1,East,SuperMart,2017,1,Widgets,"44,11,650.00","45,00,000.00"
2,South,SuperMart,2017,1,Widgets,"49,12,716.00","49,20,000.00"
3,West,SuperMart,2017,1,Widgets,"73,51,370.00","71,50,000.00"
4,North,BigCo,2017,1,Widgets,"79,66,500.00","81,50,000.00"


### 2.2. Drop Brand and UPC columns in Retailer dataframe

In [10]:
df_me=pd.DataFrame(df_me_trial)
df_me=df_me.drop(['Brand','UPC'],axis=1)

### 2.3. Reset index of dataframe

In [11]:
df_me=df_me.reset_index(drop=True)
df_me.head()

,CPG Region,Retailer,Year,Week,Category,Plan numbers,Actual numbers
0,North,SuperMart,2017,1,Widgets,707800.0,714878.0
1,East,SuperMart,2017,1,Widgets,887100.0,895971.0
2,South,SuperMart,2017,1,Widgets,2414700.0,2438847.0
3,West,SuperMart,2017,1,Widgets,1292000.0,1304920.0
4,North,BigCo,2017,1,Widgets,295500.0,298455.0


### 3.3. Sales conversion
#### 3.3.1. Function to convert value to indian currency

In [12]:
def currency_in_indian_format(n):
    """ Convert a number (int / float) into indian formatting style """
    d = decimal.Decimal(str(n))

    if d.as_tuple().exponent < -2:
        s = str(n)
    else:
        s = '{0:.2f}'.format(n)

    l = len(s)
    i = l - 1

    res, flag, k = '', 0, 0
    while i >= 0:
        if flag == 0:
            res += s[i]
            if s[i] == '.':
                flag = 1
        elif flag == 1:
            k += 1
            res += s[i]
            if k == 3 and i - 1 >= 0:
                res += ','
                flag = 2
                k = 0
        else:
            k += 1
            res += s[i]
            if k == 2 and i - 1 >= 0:
                res += ','
                flag = 2
                k = 0
        i -= 1

    return res[::-1]

#### 3.3.2. Converting Plan numbers column to indian currency

In [13]:
for i in range (0,len(df_me.index)):
    a=df_me['Plan numbers'][i]
    b=currency_in_indian_format(a)
    df_me['Plan numbers'][i]=b

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


#### 3.3.3. Converting Actual numbers column to indian currency

In [14]:
for i in range (0,len(df_me.index)):
    a=df_me['Actual numbers'][i]
    b=currency_in_indian_format(a)
    df_me['Actual numbers'][i]=b

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### 3.4. Merge both dataframes

In [15]:
df_mevsmarket=pd.merge(df_me, df_market, on=("CPG Region","Year","Week",
                                                   "Category","Retailer"))
df_mevsmarket.head()

,CPG Region,Retailer,Year,Week,Category,Plan numbers,Actual numbers,Market Plan numbers,Market Actual numbers
0,North,SuperMart,2017,1,Widgets,"7,07,800.00","7,14,878.00","10,22,500.00","10,00,000.00"
1,East,SuperMart,2017,1,Widgets,"8,87,100.00","8,95,971.00","44,11,650.00","45,00,000.00"
2,South,SuperMart,2017,1,Widgets,"24,14,700.00","24,38,847.00","49,12,716.00","49,20,000.00"
3,West,SuperMart,2017,1,Widgets,"12,92,000.00","13,04,920.00","73,51,370.00","71,50,000.00"
4,North,BigCo,2017,1,Widgets,"2,95,500.00","2,98,455.00","79,66,500.00","81,50,000.00"


## 4. Export as csv

In [16]:
df_mevsmarket.to_csv("mevsmarket.csv",index = False)